In [1]:
import pandas as pd
df =pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/bicycle/seoul_bi.csv')
df.head()

,대여일자,대여시간,대여소번호,대여구분코드,성별,연령대코드,이용건수,운동량,탄소량,이동거리,사용시간
0,2021-06-01,0,3541,정기권,F,~10대,1,0.00,0.00,0.00,8
1,2021-06-01,0,765,정기권,F,~10대,1,27.21,0.35,1526.81,19
2,2021-06-01,0,2637,정기권,F,~10대,1,41.40,0.37,1608.56,18
3,2021-06-01,0,2919,정기권,F,~10대,1,0.00,0.00,0.00,75
4,2021-06-01,0,549,정기권,F,~10대,1,13.04,0.17,731.55,6


In [2]:
## 대여일자별 데이터의 수를 데이터프레임으로 출력하고, 가장 많은 데이터가 있는 날짜를 출력하라

df['데이터개수'] = 1
result1 = df.groupby('대여일자').count().reset_index()[['대여일자', '데이터개수']]
print(result1)
print()
result2 = result1[result1['데이터개수'] == result1['데이터개수'].max()]['대여일자']
print(result2)

          대여일자  데이터개수
0   2021-06-01  48081
1   2021-06-02  56100
2   2021-06-03  18985
3   2021-06-04  56788
4   2021-06-05  52730
5   2021-06-06  47787
6   2021-06-07  52623
7   2021-06-08  55623
8   2021-06-09  54507
9   2021-06-10  36992
10  2021-06-11  52089
11  2021-06-12  48569
12  2021-06-13  43131
13  2021-06-14  54945

3    2021-06-04
Name: 대여일자, dtype: object


In [3]:
## 각 일자의 요일을 표기하고 (‘Monday’ ~’Sunday’) ‘day_name’컬럼을 추가하고 이를 이용하여 각 요일별 이용 횟수의 총합을 데이터 프레임으로 출력하라

def week_name(x):
    week_list = ['Monday' ,'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
    tmp = x.weekday()
    return week_list[tmp]

df['day_name'] = pd.to_datetime(df['대여일자']).apply(week_name)
df.groupby('day_name').count()['데이터개수']

day_name
Friday       108877
Monday       107568
Saturday     101299
Sunday        90918
Thursday      55977
Tuesday      103704
Wednesday    110607
Name: 데이터개수, dtype: int64

In [4]:
## 각 요일별 가장 많이 이용한 대여소의 이용횟수와 대여소 번호를 데이터 프레임으로 출력하라

tmp = df.groupby(['day_name', '대여소번호']).size().to_frame('size').sort_values(by=['day_name', 'size'], ascending=False).reset_index() 
result = tmp.drop_duplicates('day_name', keep='first')
print(result)

        day_name  대여소번호  size
0      Wednesday    502   282
2402     Tuesday    502   267
4801    Thursday   2715   137
7197      Sunday    502   372
9611    Saturday    502   378
12031     Monday    502   242
14453     Friday    502   277


In [5]:
df['대여구분코드'].value_counts()

정기권         555593
일일권         120328
단체권           3018
일일권(비회원)        11
Name: 대여구분코드, dtype: int64

In [6]:
## 나이대별 대여구분 코드의 (일일권/전체횟수) 비율을 구한 후 가장 높은 비율을 가지는 나이대를 확인하라. 일일권의 경우 일일권 과 일일권(비회원)을 모두 포함하라

# 일일권 개수 구하기
tmp1 = df[(df['대여구분코드'] == '일일권') | (df['대여구분코드'] == "일일권(비회원)")] 
tmp1 = tmp1.groupby(['연령대코드']).size().to_frame('일일권').reset_index()

# 전체 개수 구하기
tmp2 = df.groupby(['연령대코드']).size().to_frame('total').reset_index()
tmp2['일일권'] = tmp1['일일권']
tmp2['일일권_ratio'] = tmp2['일일권'] / tmp2['total']

result = tmp2.sort_values(by='일일권_ratio', ascending=False).iloc[0]['연령대코드']
print(result)

~10대


In [7]:
## 연령대별 평균 이동거리를 구하여라

result = df.groupby(['연령대코드']).mean().reset_index()[['연령대코드', '이동거리']]
print(result)

  연령대코드         이동거리
0   20대  3211.890552
1   30대  3341.443859
2   40대  3514.857416
3   50대  3593.668100
4   60대  3538.145737
5  70대~  3085.039641
6  ~10대  2634.426279


In [8]:
## 연령대 코드가 20대인 데이터를 추출하고,이동거리값이 추출한 데이터의 이동거리값의 평균 이상인 데이터를 추출한다.
## 최종 추출된 데이터를 대여일자, 대여소 번호 순서로 내림차순 정렬 후 1행부터 200행까지의 탄소량의 평균을 소숫점 3째 자리까지 구하여라

tmp20 = df[df['연령대코드'] == '20대'] 
tmp20_1 = tmp20[tmp20['이동거리'] >= tmp20['이동거리'].mean()]

tmp20_2 = tmp20_1.sort_values(by=['대여일자', '대여소번호'], ascending=False).head(200)
result = round(tmp20_2['탄소량'].astype(float).mean(), 3)
print(result)

1.613


In [9]:
## 6월 7일 ~10대의 “이용건수”의 중앙값은?

result = df[(df['대여일자']=='2021-06-07') & (df['연령대코드']=='~10대')]['이용건수'].median()
print(result)

1.0


In [10]:
## 평일 (월~금) 출근 시간대(오전 6,7,8시)의 대여소별 이용 횟수를 구해서 데이터 프레임 형태로 표현한 후 각 대여시간별 이용 횟수의 상위 3개 대여소와 이용횟수를 출력하라

tmp = df[(df['day_name']!='Saturday') & (df['day_name']!='Sunday') & (df['대여시간']>=6) & (df['대여시간']<=8)]
tmp2 = tmp.groupby(['대여시간', '대여소번호']).count()['이용건수'].to_frame('이용횟수').reset_index()
tmp2.sort_values(by=['대여시간', '이용횟수'], ascending=False).groupby('대여시간').head(3)

,대여시간,대여소번호,이용횟수
5671,8,2701,119
4391,8,646,115
4748,8,1152,92
1893,7,259,104
1865,7,230,77
2245,7,726,77
1367,6,2744,45
626,6,1125,40
562,6,1028,36


In [11]:
## 이동거리의 평균 이상의 이동거리 값을 가지는 데이터를 추출하여 추출데이터의 이동거리의 표본표준편차 값을 구하여라

tmp = df[df['이동거리'] >= df['이동거리'].mean()]
tmp['이동거리'].std()

5092.139707505305

In [12]:
## 남성(‘M’ or ‘m’)과 여성(‘F’ or ‘f’)의 이동거리값의 평균값을 구하여라

for i in range(len(df)):
    if df['성별'].iloc[i] == 'm':
        df['성별'].iloc[i] = 'M'
    elif df['성별'].iloc[i] == 'f':
        df['성별'].iloc[i] = 'F'

result = df.groupby(['성별']).mean()['이동거리']
print(result)

C:\Users\ccmedia\AppData\Local\Temp\ipykernel_9996\3536601244.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['성별'].iloc[i] = 'M'
C:\Users\ccmedia\AppData\Local\Temp\ipykernel_9996\3536601244.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['성별'].iloc[i] = 'F'


성별
F    3468.575025
M    3209.110871
Name: 이동거리, dtype: float64
